In [29]:
#Код держится на молитвах 😭😭😭😭

import tkinter as tk
import random
import requests
import re
from textblob import TextBlob

def load_text():
    try:
        url = "https://www.gutenberg.org/files/514/514-0.txt"
        response = requests.get(url)
        if response.status_code != 200:
            return None, f"Error: Failed to load text, status code {response.status_code}"
        text = response.text
        cleaned_text = re.sub(r'\s+', ' ', text).strip()
        return cleaned_text, "Text loaded successfully."
    except Exception as e:
        return None, f"Error: {str(e)}"

def split_into_sentences(text):
    sentence_endings = re.compile(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s')
    sentences = sentence_endings.split(text)
    return [s.strip() for s in sentences if s.strip()]

def get_random_sentences():
    if not full_text:
        return ["Error: No text loaded."]
    sentences = split_into_sentences(full_text)
    if not sentences:
        return ["Error: No sentences found."]
    return random.sample(sentences, min(10, len(sentences)))

def show_sentences():
    sentence_text.delete(1.0, tk.END)
    sentences = get_random_sentences()
    for sentence in sentences:
        sentence_text.insert(tk.END, sentence + "\n")
    extract_characters(sentences)

def highlight_a_sentences():
    a_sentences = []
    for i, sentence in enumerate(sentence_text.get(1.0, tk.END).splitlines()):
        if sentence.startswith("A"):
            a_sentences.append(sentence)
            sentence_text.tag_add("highlight_a", f"{i+1}.0", f"{i+1}.end")
    if a_sentences:
        a_sentence_text.delete(1.0, tk.END)
        a_sentence_text.insert(tk.END, "\n".join(a_sentences))
        sentence_text.see(sentence_text.index(f"{a_sentences.index(a_sentences[0])+2}.0"))
    else:
        a_sentence_text.delete(1.0, tk.END)
        a_sentence_text.insert(tk.END, "No sentences starting with 'A' found.")

def extract_characters(sentences):
    character_names = set()
    for sentence in sentences:
        blob = TextBlob(sentence)
        for word, pos in blob.tags:
            if pos == 'NNP' and len(word) > 1:
                character_names.add(word)
    character_text.delete(1.0, tk.END)
    if character_names:
        character_text.insert(tk.END, "\n".join(character_names))
    else:
        character_text.insert(tk.END, "No character names found in these sentences.")

def show_synonym():
    try:
        selected_text = sentence_text.get(tk.SEL_FIRST, tk.SEL_LAST)
        blob = TextBlob(selected_text)
        synonyms = []
        for word in blob.words:
            if word.synsets:
                syns = word.synsets[0].lemma_names()
                synonyms.extend(syns)
        result_label.config(text=f"Synonyms: {', '.join(set(synonyms))}")
    except tk.TclError:
        result_label.config(text="Please select a word.")

def show_antonym():
    try:
        selected_text = sentence_text.get(tk.SEL_FIRST, tk.SEL_LAST)
        blob = TextBlob(selected_text)
        antonyms = []
        for word in blob.words:
            for syn in word.synsets:
                for lemma in syn.lemmas():
                    antonyms.extend(ant.name() for ant in lemma.antonyms())
        unique_antonyms = set(antonyms)
        result_label.config(text=f"Antonyms: {', '.join(unique_antonyms) if unique_antonyms else 'No antonyms found.'}")
    except tk.TclError:
        result_label.config(text="Please select a word.")

def show_word_count():
    text_content = sentence_text.get(1.0, tk.END).strip()
    words = text_content.split()
    word_count = len(words)
    result_label.config(text=f"Word Count: {word_count}")

def show_lemmas():
    try:
        selected_text = sentence_text.get(tk.SEL_FIRST, tk.SEL_LAST)
        blob = TextBlob(selected_text)
        lemmas = [word.lemma for word in blob.words]
        result_label.config(text=f"Lemmas: {', '.join(set(lemmas))}")
    except tk.TclError:
        result_label.config(text="Please select a word.")

full_text, status_message = load_text()
print(status_message)

root = tk.Tk()
root.title("Working with TextBlob")

title_label = tk.Label(root, text="Working with TextBlob", font=("Arial", 16))
title_label.grid(row=0, column=1, columnspan=2)

sentence_label = tk.Label(root, text="10 random sentences from Little Women")
sentence_label.grid(row=1, column=0)
sentence_text = tk.Text(root, height=10, width=60)
sentence_text.grid(row=2, column=0, columnspan=2)

a_sentence_label = tk.Label(root, text="Sentences starting with 'A'")
a_sentence_label.grid(row=3, column=0)
a_sentence_text = tk.Text(root, height=5, width=60, bg="light cyan")
a_sentence_text.grid(row=4, column=0, columnspan=2)

sentence_text.tag_config("highlight_a", foreground="blue")

select_button = tk.Button(root, text="Show Sentences", command=show_sentences)
select_button.grid(row=1, column=2)

highlight_button = tk.Button(root, text="Highlight 'A' Sentences", command=highlight_a_sentences)
highlight_button.grid(row=3, column=2)

character_label = tk.Label(root, text="The list of the characters from Little Women")
character_label.grid(row=6, column=0)
character_text = tk.Text(root, height=5, width=50)
character_text.grid(row=7, column=0, columnspan=2)

synonym_button = tk.Button(root, text="Synonym", command=show_synonym)
synonym_button.grid(row=8, column=0)

antonym_button = tk.Button(root, text="Antonym", command=show_antonym)
antonym_button.grid(row=8, column=1)

lemmas_button = tk.Button(root, text="Lemmas", command=show_lemmas)
lemmas_button.grid(row=8, column=2)

words_button = tk.Button(root, text="Words", command=show_word_count)
words_button.grid(row=9, column=2)

result_label = tk.Label(root, text="")
result_label.grid(row=10, column=0, columnspan=3)

ok_button = tk.Button(root, text="OK")
ok_button.grid(row=11, column=1)

cancel_button = tk.Button(root, text="Cancel", command=root.quit)
cancel_button.grid(row=11, column=2)

root.mainloop()


Text loaded successfully.
